In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

We will use Machine Learning Explainability strategies here to find out which features impact the model the most.
Looking at permutation importance first, we will separate the categorical features now and deal with them later.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
import pandas as pd
from sklearn.metrics import mean_absolute_error


train_data = pd.read_csv("../input/home-data-for-ml-course/train.csv", index_col = 'Id')
test_data = pd.read_csv("../input/home-data-for-ml-course/test.csv", index_col = 'Id')
print("Data Loaded Successfully")


y = train_data.SalePrice.reset_index(drop=True)
train_data.drop(['SalePrice'], axis=1, inplace = True)

In [ ]:
#Dropping valyes with high null values
train_data.drop(['Alley', 'PoolQC', 'MiscFeature', 'PoolArea', 'Fence'], axis = 1, inplace=True)
test_data.drop(['Alley', 'PoolQC', 'MiscFeature', 'PoolArea', 'Fence'], axis = 1, inplace=True)

In [ ]:
for column in train_data:
    if train_data[column].isna().any():
        print('{0} has {1} null values'.format(column, train_data[column].isnull().sum()))

In [ ]:
#Number features that should be string features
train_data['MSSubClass'] = train_data['MSSubClass'].apply(str)
test_data['MSSubClass'] = test_data['MSSubClass'].apply(str)

train_data['YrSold'] = train_data['YrSold'].astype(str)
test_data['YrSold'] = test_data['YrSold'].astype(str)

train_data['MoSold'] = train_data['MoSold'].astype(str)
test_data['MoSold'] = test_data['MoSold'].astype(str)

In [ ]:
#Filling in NA values

for feature in ['Functional', 'Electrical', 'KitchenQual', 'Exterior1st', 'Exterior2nd', 'SaleType']:
    train_data[feature] = train_data[feature].fillna(train_data[feature].mode()[0])
    test_data[feature] = test_data[feature].fillna(test_data[feature].mode()[0])

for col in ('GarageYrBlt', 'GarageArea', 'GarageCars', 'MasVnrArea'):
    train_data[col] = train_data[col].fillna(0)
    test_data[col] = test_data[col].fillna(0)
for col in ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'MasVnrType', 'FireplaceQu']:
    train_data[col] = train_data[col].fillna('None')
    test_data[col] = test_data[col].fillna('None')

#Grouping lot frontage by neighborhood seems to make sense, and filling in any NA's with median
train_data['LotFrontage'] = train_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))
test_data['LotFrontage'] = test_data.groupby('Neighborhood')['LotFrontage'].transform(lambda x: x.fillna(x.median()))

#Feature Engineering. ie. combining logical features together
train_data['YrBltAndRemod'] = train_data['YearBuilt'] + train_data['YearRemodAdd']
test_data['YrBltAndRemod'] = test_data['YearBuilt'] + test_data['YearRemodAdd']

train_data['TotalSF'] = train_data['TotalBsmtSF'] + train_data['1stFlrSF'] + train_data['2ndFlrSF']
test_data['TotalSF'] = test_data['TotalBsmtSF'] + test_data['1stFlrSF'] + test_data['2ndFlrSF']

train_data['TotalFinSF'] = (train_data['BsmtFinSF1'] + train_data['BsmtFinSF2'] + train_data['1stFlrSF'] + train_data['2ndFlrSF'])
test_data['TotalFinSF'] = (test_data['BsmtFinSF1'] + test_data['BsmtFinSF2'] + test_data['1stFlrSF'] + test_data['2ndFlrSF'])

train_data['Total_Bathrooms'] = (train_data['FullBath'] + (0.5 * train_data['HalfBath']) + train_data['BsmtFullBath'] + (0.5 * train_data['BsmtHalfBath']))
test_data['Total_Bathrooms'] = (test_data['FullBath'] + (0.5 * test_data['HalfBath']) + test_data['BsmtFullBath'] + (0.5 * test_data['BsmtHalfBath']))

train_data['Total_porch_sf'] = (train_data['OpenPorchSF'] + train_data['3SsnPorch'] + train_data['EnclosedPorch'] + train_data['ScreenPorch'] + train_data['WoodDeckSF'])
test_data['Total_porch_sf'] = (test_data['OpenPorchSF'] + test_data['3SsnPorch'] + test_data['EnclosedPorch'] + test_data['ScreenPorch'] + test_data['WoodDeckSF'])

#Drop old features
train_data.drop(['YearBuilt', 'YearRemodAdd'], axis=1, inplace=True)
test_data.drop(['YearBuilt', 'YearRemodAdd'], axis=1, inplace=True)
train_data.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
test_data.drop(['TotalBsmtSF', '1stFlrSF', '2ndFlrSF'], axis=1, inplace=True)
train_data.drop(['BsmtFinSF1', 'BsmtFinSF2'], axis=1, inplace=True)
test_data.drop(['BsmtFinSF1', 'BsmtFinSF2'], axis=1, inplace=True)
train_data.drop(['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath'], axis=1, inplace=True)
test_data.drop(['FullBath', 'HalfBath', 'BsmtFullBath', 'BsmtHalfBath'], axis=1, inplace=True)
train_data.drop(['OpenPorchSF', '3SsnPorch', 'EnclosedPorch', 'ScreenPorch', 'WoodDeckSF'], axis=1, inplace=True)
test_data.drop(['OpenPorchSF', '3SsnPorch', 'EnclosedPorch', 'ScreenPorch', 'WoodDeckSF'], axis=1, inplace=True)


for column in train_data:
    if train_data[column].isna().any():
        print('{0} has {1} null values'.format(column, train_data[column].isnull().sum()))

#train_data.columns

In [ ]:
num_features = [i for i in train_data.columns if train_data[i].dtype in ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']]
cat_features_small = [i for i in train_data.columns if train_data[i].dtype == 'object' and train_data[i].nunique() <= 15]
cat_features_large = [i for i in train_data.columns if train_data[i].dtype == 'object' and train_data[i].nunique() > 15]

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
import category_encoders as ce
from xgboost import XGBRegressor

#Preprocessing numerical data
numerical_transformer = Pipeline(verbose=False,steps=[
    ('imputer_num', SimpleImputer(strategy='median'))
])

#Preprocessing categorical data
categorical_onehot_transformer = Pipeline(verbose=False,steps=[
    ('imputer_onehot', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

#Preprocessing ordinal cat data (ie. clear ordering to values (3 is greater than 2 is greater than 1, etc.))
categorical_label_transformer = Pipeline(verbose=False, steps=[
    ('imputer_label', SimpleImputer(strategy='most_frequent')),
    ('label', ce.OrdinalEncoder(handle_unknown='ignore'))
])

categorical_count_transformer = Pipeline(verbose=False, steps=[
    ('imputer_label', SimpleImputer(strategy = 'most_frequent')),
    ('count', ce.TargetEncoder(handle_unknown = 'count'))
])

preprocessor = ColumnTransformer(verbose=False,
    transformers = [
        ('num', numerical_transformer, num_features),
        #('cox_box', PowerTransformer(method='yeo-johnson', standardize=False),skew_cols),
        #('cat_label', categorical_label_transformer, categorical_label_cols),
        ('cat_onehot', categorical_onehot_transformer, cat_features_small),
        ('cat_count', categorical_count_transformer, cat_features_large),
    ])

#Here we are using StandardScaler, which subtracts the mean from each features and then scale to unit variance.
train_pipeline = Pipeline(verbose=False,steps=[
                    ('preprocessor', preprocessor),   
                    ('scale', StandardScaler(with_mean=False,with_std=True)),
                    ('model', XGBRegressor(random_state=0))
                    ])

In [ ]:
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"
import eli5
from eli5.sklearn import PermutationImportance

pd.set_option('display.max_columns', 500)


X_train, X_val, y_train, y_val = train_test_split(train_data[num_features], y, train_size = 0.8, test_size = 0.2, random_state = 0)

#Base Model MAE w/ just numerical data is 15,347 for reference
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train,
            early_stopping_rounds=5, 
             eval_set=[(X_val, y_val)], 
             verbose=False)

perm = PermutationImportance(my_model, random_state=1).fit(X_val, y_val)
eli5.show_weights(perm, feature_names = X_val.columns.tolist(), top = None)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV

param_grid = {'model__nthread':[2], #when use hyperthread, xgboost may become slower
              'model__learning_rate': [0.04, 0.05], #so called `eta` value
              'model__max_depth': range(3,5,1),
#               'model__importance_type': ['weight', 'gain', 'cover'],
#               "model__min_child_weight" : [ 1 ],
#               "model__gamma"            : [ 0.0],
              "model__colsample_bytree" : [ 0.2 ],
              'model__silent': [1],
              'model__n_estimators': [700], #number of trees
#               'model__n_estimators': range(595,600,1), #number of trees
#               'model__n_estimators': range(550,1000,5), #number of trees
             }
searched_model = GridSearchCV(estimator=train_pipeline,param_grid = param_grid, scoring="neg_mean_absolute_error", cv=5, error_score='raise', verbose = 1)
searched_model.fit(train_data,y)

print(searched_model.best_estimator_)
print(searched_model.best_score_)

In [ ]:
#import shap
#shap.initjs()

#explainer = shap.TreeExplainer(searched_model)
#shap_values = explainer.shap_values(X_val)

# visualize the first prediction's explanation (use matplotlib=True to avoid Javascript)
#shap.force_plot(explainer.expected_value, shap_values[201,:], X_val.iloc[201,:])

In [ ]:
#shap.summary_plot(shap_values, val_X)

In [ ]:
preds_test = searched_model.predict(test_data)
# Save test predictions to file
output = pd.DataFrame({'Id': test_data.index,'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)
output